In [4]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
import math

import warnings; warnings.simplefilter('ignore')

In [5]:
# sp500_url = "https://raw.githubusercontent.com/saikr789/stock-index-risk/master/SP500companies.csv"

In [8]:
# sp500 = pd.read_csv(sp500_url)
sp500 = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500companies.csv")
sp500_sc = list(sp500['Security Code'])
sp500_sn = list(sp500['Security Name'])
sp500_companies = {str(sp500_sc[i]) : sp500_sn[i] for i in range(len(sp500_sc))}
# sp500_companies

In [9]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

In [10]:
# cols = ["Company", "Security Id", "Actual Average Returns %", "Predicted Negative Range", "Predicted Positive Range", "Prediction Range"]
# results_df = pd.DataFrame(columns = cols)
# results_df

In [11]:
%%time

def getAverageResults(data, company, sec_id, days, results_df):
    try :
        df = data[data["Security Id"] == sec_id]
        if (df.shape[0] != 0):
            df1 = df[df["Difference"] < 0]
            df2 = df[df["Difference"] > 0]
            actual_average_returns = df["Actual average returns %"].sum() / df.shape[0]
            if (actual_average_returns == 0):
                row = {}
                row["Company"] = company
                row["Security Id"] = sec_id
                row["Days"] = days
                row["Actual Average Returns %"] = actual_average_returns
                row["Predicted Negative Range"] = 0
                row["Predicted Positive Range"] = 0
                row["Prediction Range"] = "-"
            else:
                predicted_negative_range = df1["Difference"].sum() / df1.shape[0]
                predicted_positive_range = df2["Difference"].sum() / df2.shape[0]
                if (math.isnan(predicted_negative_range)):
                    predicted_negative_range = actual_average_returns
                    range1 = actual_average_returns
                    range2 = (1 + predicted_positive_range) * actual_average_returns
                elif (math.isnan(predicted_positive_range)):
                    predicted_positive_range = actual_average_returns
                    range1 = (1- abs(predicted_negative_range)) * actual_average_returns
                    range2 = actual_average_returns
                else:
                    range1 = (1- abs(predicted_negative_range)) * actual_average_returns
                    range2 = (1 + predicted_positive_range) * actual_average_returns
    #             print(range1, range2)
                row = {}
                row["Company"] = company
                row["Security Id"] = sec_id
                row["Days"] = days
                row["Actual Average Returns %"] = actual_average_returns
                row["Predicted Negative Range"] = predicted_negative_range
                row["Predicted Positive Range"] = predicted_positive_range
                row["Prediction Range"] = str(range1) + " to " + str(range2)
            results_df = results_df.append(row, ignore_index=True)
    except Exception as e:
        print(e)

    return results_df

    

Wall time: 0 ns


In [12]:
# days = [30, 60, 90, 180, 270, 360, 540, 720, 900, 1080]
# days = [30, 60, 90, 360, 720, 180, 1080]
days = [30, 60, 90, 180, 360, 720]

for d in days:
    print(d)
    print("*****************************************************************")
    
    cols = ["Company", "Security Id", "Days", "Actual Average Returns %", "Predicted Negative Range", "Predicted Positive Range", "Prediction Range"]
    results_df = pd.DataFrame(columns = cols)
    
    all_results_data = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\All_" + str(d) + "days_Results (2).csv")
    
    for sec_id in sp500_sc:
        company_name = sp500_companies[str(sec_id)]
        results_df = getAverageResults(all_results_data, company_name, sec_id, d, results_df)
    
#     print(results_df)
    
    results_df = results_df.sort_values(by = ['Actual Average Returns %'], ascending = [False])
    results_df.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\Top_New_Updated_" + str(d) + "days_WithPredictionRange.csv", index = False)
    
    print("*****************************************************************")
#     break

30
*****************************************************************
*****************************************************************
60
*****************************************************************
*****************************************************************
90
*****************************************************************
*****************************************************************
180
*****************************************************************
*****************************************************************
360
*****************************************************************
*****************************************************************
720
*****************************************************************
*****************************************************************
